<a href="https://colab.research.google.com/github/Bibhukalyan/simple_RNN/blob/main/simpleRNN_using_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [83]:
# Load imdb dataset
max_features = 10000 ##vocabulary size
maxlen = 500 ##max length of each review
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

In [84]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(25000,) (25000,)
(25000,) (25000,)


In [85]:
#Inspect a sample review
print(X_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [86]:
print(y_test[0])

0


In [87]:
word_index = imdb.get_word_index()
word_index

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [88]:
#Mapping of word index back to word
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [89]:
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [90]:
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in X_train[0]])
print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [91]:
#add padding
max_lenght = 500 ##max length of each review
X_train = sequence.pad_sequences(X_train, maxlen=max_lenght)
X_test = sequence.pad_sequences(X_test, maxlen=max_lenght)

In [92]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [93]:
# train simple RNN
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_lenght)) ##embedding layer with 128 feature that will be like dimentionality reduction
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [94]:
#Create an instance of early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) ##patience is the number of epoche it will wait



In [95]:
# Train the model with early stopping
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 226ms/step - accuracy: 0.6027 - loss: 54037412.0000 - val_accuracy: 0.7888 - val_loss: 0.4488
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 228ms/step - accuracy: 0.8248 - loss: 0.4028 - val_accuracy: 0.7866 - val_loss: 0.4600
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 228ms/step - accuracy: 0.8667 - loss: 0.3153 - val_accuracy: 0.8108 - val_loss: 0.4435
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 202s 228ms/step - accuracy: 0.8991 - loss: 0.2548 - val_accuracy: 0.8108 - val_loss: 0.4815
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 201s 226ms/step - accuracy: 0.9234 - loss: 0.2249 - val_accuracy: 0.8132 - val_loss: 0.5755
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 228ms/step - accuracy: 0.9362 - loss: 0.1717 - val_accuracy: 0.7962 - val_loss: 0.5246
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 201s 227ms/step - accuracy: 0.9489 - loss: 0.1389 - val_accuracy: 0.8028 - val_loss: 0.5650
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 228ms/step - accuracy: 0

In [97]:
#model paramether details
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (32, 500, 128)              │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (32, 128)                   │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (32, 1)                     │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,939,077 (15.03 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,052 (10.02 MB)

In [98]:
model.save('simple_rnn_model_imdb.h5')

In [26]:
#creation of web app from the above model
#Mapping of word index back to word
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
#Load the pre trianind model using ReLU activation function
model = tf.keras.models.load_model('simple_rnn_model_imdb.h5')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (32, 500, 128)              │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (32, 128)                   │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (32, 1)                     │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [29]:
model.get_weights()

[array([[ 5.5537260e-01,  6.0996413e-01,  5.2372110e-01, ...,
         -5.7706052e-01, -5.9728193e-01, -5.7815063e-01],
        [-5.1638763e-04,  4.7783949e-03,  5.1706299e-02, ...,
         -1.2941369e-02,  1.2553590e-03, -3.0581649e-02],
        [-9.9891867e-04, -8.7088132e-03, -4.7856644e-03, ...,
         -1.2322286e-01, -6.7778692e-02, -7.4963681e-02],
        ...,
        [-6.6717863e-03,  2.6197713e-02, -3.0112553e-02, ...,
          1.5000737e-02, -4.5136038e-02,  1.3338616e-02],
        [ 2.5316501e-02, -6.7453332e-02, -2.9299211e-02, ...,
         -7.6328404e-02,  1.4398634e-02, -7.3575586e-02],
        [-1.4110968e-02,  2.7772600e-02,  5.5819536e-03, ...,
          8.1594512e-02, -5.2101560e-02,  1.8408505e-02]], dtype=float32),
 array([[ 0.12762135,  0.08709295, -0.04953506, ...,  0.03614141,
          0.05551203,  0.13947162],
        [ 0.15530352,  0.01908583,  0.05613916, ...,  0.0647954 ,
          0.07175933,  0.08439949],
        [-0.06437004,  0.05744495,  0.16134268

In [99]:
#decoding review
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

#to process user input
def process_review(review):
  words = review.lower().split()
  encoded_review = [word_index.get(word, 2) + 3 for word in words]
  encoded_review = sequence.pad_sequences([encoded_review], maxlen=500)
  return encoded_review




In [100]:
from os import sendfile
#prediction function
def predict_sentiment(review):
  processed_review = process_review(review)
  prediction = model.predict(processed_review)[0][0]
  sentiment = 'positive' if prediction > 0.5 else 'negative'
  return sentiment, prediction

In [102]:
#example movie review
example_review = "This moview was fantastic! The acting was great and the plot was thrilling."
sentiment, prediction = predict_sentiment(example_review)
print(f"Sentiment: {sentiment}, Prediction: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Sentiment: positive, Prediction: 0.9974029064178467


In [103]:
decoded_review = decode_review(process_review(example_review)[0])
decoded_review

'? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this and was and the acti

In [104]:
print(process_review(example_review))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [ ]:
%%writefile app.py
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
import streamlit as st

#load imdb word index and reverse word index
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Load the pre-trained model
model = load_model('simple_rnn_model_imdb.h5')

#decode the review text
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

#encode the review
def process_review(review):
    words = review.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    encoded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return encoded_review

#sentiment prediction of moview
def predict_sentiment(review):
    processed_review = process_review(review)
    prediction = model.predict(processed_review)[0][0]
    sentiment = 'positive' if prediction > 0.5 else 'negative'
    return sentiment, prediction


#streamlit app UI
st.title("IMDb Movie Review Sentiment Analysis")
st.write("Enter a movie review to predict its sentiment.")

user_input = st.text_input("Enter a movie review:")

if st.button("Analyze"):
    if user_input:
        sentiment, prediction = predict_sentiment(user_input)
        st.write(f"Sentiment: {sentiment}")
        st.write(f"Prediction Score: {prediction}")
    else:
        st.warning("Please enter a movie review.")

